In [1]:
import collections
import sys
import numpy
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, SimpleRNN

Using TensorFlow backend.


In [2]:
from keras.layers import Merge
from keras.layers import Lambda
from keras import backend as K
import keras

In [3]:
questions = open('abstract_math_questions.txt','r')
labels = open('abstract_math_labels.txt','r')
info_sentences = open('abstract_math_info.txt','r')
fact_sentences = open('abstract_math_facts.txt','r')
standard_relations = open('abstract_math_relations.txt','r')

questions_list = []
labels_list = []
info_list = []
relations_list = []
facts_list = []

for sentence in questions:
    questions_list.append(sentence)
    
for sentence in labels:
    labels_list.append(sentence)
    
for sentence in info_sentences:
    info_list.append(sentence)

for sentence in fact_sentences:
    facts_list.append(sentence)
    
for sentence in standard_relations:
    relations_list.append(sentence)

In [4]:
print(len(questions_list[0]))
print(len(labels_list[0]))
print(len(info_list[0]))
print(len(facts_list[0]))
print(len(relations_list[0]))

24
10
26
5
30


In [5]:
#in this version, we put placeholders for numbers, such as QN1, QN2, etc.. for question numbers

fact_num_dict_list = []
ques_num_dict_list = []
info_num_dict_list = []

#store the numbers in a list of dicts
#for facts list:
for sentence in facts_list:
    fact_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            fact_num_dict[word] = "FN%d".format(indx_num)
            word = "FN%d".format(indx_num)
            indx_num += 1
    fact_num_dict_list.append(fact_num_dict)

#repeat for questions
for sentence in questions_list:
    quest_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            quest_num_dict[word] = "QN%d".format(indx_num)
            word = "QN%d".format(indx_num)
            indx_num += 1
    ques_num_dict_list.append(quest_num_dict)   

#repeat for information
for sentence in info_list:
    info_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            info_num_dict[word] = "IN%d".format(indx_num)
            word = "IN%d".format(indx_num)
            indx_num += 1
    info_num_dict_list.append(info_num_dict)   

#relations do not need since it will already be in encoded format

In [6]:
list_of_all_variables = []

### Let's make a list of all variables and prepare them for one-hot-coding

# Start by assigning numbers to the standard variables:

list_of_all_variables.append("QN1")
list_of_all_variables.append("QN2")
list_of_all_variables.append("QN3")
list_of_all_variables.append("QN4")
list_of_all_variables.append("QN5")
list_of_all_variables.append("QN6")
list_of_all_variables.append("QN7")
list_of_all_variables.append("QN8")
list_of_all_variables.append("FN1")
list_of_all_variables.append("FN2")
list_of_all_variables.append("FN3")
list_of_all_variables.append("FN4")
list_of_all_variables.append("FN5")
list_of_all_variables.append("FN6")
list_of_all_variables.append("FN7")
list_of_all_variables.append("FN8")
list_of_all_variables.append("IN1")
list_of_all_variables.append("N2")
list_of_all_variables.append("IN3")
list_of_all_variables.append("IN4")
list_of_all_variables.append("IN5")
list_of_all_variables.append("IN6")
list_of_all_variables.append("IN7")
list_of_all_variables.append("IN8")
list_of_all_variables.append("N1")
list_of_all_variables.append("N2")
list_of_all_variables.append("N3")
list_of_all_variables.append("N4")
list_of_all_variables.append("N5")
list_of_all_variables.append("N6")
list_of_all_variables.append("N7")
list_of_all_variables.append("N8")
list_of_all_variables.append("N9")
list_of_all_variables.append("V1")
list_of_all_variables.append("V2")
list_of_all_variables.append("V3")
list_of_all_variables.append("V4")
list_of_all_variables.append("V5")
list_of_all_variables.append("V6")
list_of_all_variables.append("V7")
list_of_all_variables.append("V8")
list_of_all_variables.append("V9")

for sentence in facts_list:
    for word in sentence.split():
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)
    
for sentence in info_list:
    for word in sentence.split():
        if word not in list_of_all_variables:
            list_of_all_variables.append(word) 

for sentence in questions_list:
    for word in sentence.split():
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)

for sentence in relations_list:
    for word in sentence.split():
        if word not in list_of_all_variables:
            list_of_all_variables.append(word)      
            
list_of_all_variables.append("<PAD>")

In [7]:
word_to_index = {}
index_to_word = {}

word_index = 0

for word in list_of_all_variables:
    word_to_index[word] = word_index
    index_to_word[word_index] = word
    word_index += 1
print(list_of_all_variables)

['QN1', 'QN2', 'QN3', 'QN4', 'QN5', 'QN6', 'QN7', 'QN8', 'FN1', 'FN2', 'FN3', 'FN4', 'FN5', 'FN6', 'FN7', 'FN8', 'IN1', 'N2', 'IN3', 'IN4', 'IN5', 'IN6', 'IN7', 'IN8', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'NONE', 'MEANS', 'MORE', 'return', 'MAKES', 'ADD', ';', 'buy', 'SUBTRACT', 'changed', 'PAST-TENSE', 'change', 'ADD-OR-SUBTRACT', 'PLURAL', 'A1', 'opened', 'open', 'IN', 'HAVE', 'there', 'are', '6', 'and', '2', '.', 'in', 'the', ',', '4', 'UNITS', 'of', '7', '5', '9', '8', 'at', 'with', 'next', 'to', '3', 'on', 'other', 'side', 'from', 'has', 'already', 'one', 'were', 'a', 'comes', 'he', 'bought', 'last', 'week', 'yesterday', 'called', 'add', 'more', 'subtract', 'went', 'could', 'not', 'go', 'for', 'as', 'another', 'each', 'some', 'first', 'they', '12', '36', 'second', '25', '24', 'during', 'had', '40', '47', 'month', '16', '19', 'than', '60', '34', 'men', '45', 'women', '18', 'days', '21', 'it', 'will', 'be', '15',

In [8]:
new_facts_list = []
new_questions_list = []
new_info_list = []
new_relations_list = []

pad_up_to = 100
pad_index = word_to_index["<PAD>"]

for sentence in facts_list:
    substitute_sentence = []
    for word in sentence.split():
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_facts_list.append(substitute_sentence)

for sentence in questions_list:
    substitute_sentence = []
    for word in sentence.split():
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_questions_list.append(substitute_sentence)
        
for sentence in info_list:
    substitute_sentence = []
    for word in sentence.split():
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_info_list.append(substitute_sentence)

for sentence in relations_list:
    substitute_sentence = []
    for word in sentence.split():
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_relations_list.append(substitute_sentence)

In [9]:
print(len(new_facts_list))
print(len(new_questions_list))
print(len(new_info_list))
print(len(new_relations_list))

79
79
79
79


In [10]:
one_hot_facts = []
one_hot_questions = []
one_hot_info = []
one_hot_relations = []

one_hot_length = len(word_to_index)

for sentence in new_facts_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_facts.append(substitute_sentence)

for sentence in new_questions_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_questions.append(substitute_sentence)

for sentence in new_info_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_info.append(substitute_sentence)
    
for sentence in new_relations_list:
    substitute_sentence = []
    for word_ind in sentence:
        substitute_word = []
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_word.append(1)
            else:
                substitute_word.append(0)
        substitute_sentence.append(substitute_word)
    one_hot_relations.append(substitute_sentence)
    
vectorized_fact_list = np.asarray(one_hot_facts)
vectorized_questions = np.asarray(one_hot_questions) 
vectorized_info = np.asarray(one_hot_info)
vectorized_relations = np.asarray(one_hot_relations)

In [11]:
print(vectorized_fact_list.shape)
print(vectorized_questions.shape)
print(vectorized_info.shape)
print(vectorized_relations.shape)

(79, 100, 168)
(79, 100, 168)
(79, 100, 168)
(79, 100, 168)


In [12]:
label_words_to_index = {}
label_index_to_words = {}
label_index = 0
label_words_to_index["\t"]=label_index
label_index_to_words[label_index]="\t"
label_index += 1
label_words_to_index["<PAD>"]=label_index
label_index_to_words[label_index]="<PAD>"
new_label_list = []
label_pad_up_to = 50
for sentence in labels_list:
    substitute_sentence = []
    sent = sentence.split()
    for char in sent: #char could be a word or group of characters
        if char in label_words_to_index.keys():
            substitute_sentence.append(label_words_to_index[char])
        else:
            label_index += 1
            label_words_to_index[char]=label_index
            label_index_to_words[label_index]=char
            substitute_sentence.append(label_words_to_index[char])
    for i in range(len(substitute_sentence),label_pad_up_to): #pad to regular
        substitute_sentence.append(label_words_to_index["<PAD>"])
        
    new_label_list.append(substitute_sentence)
    
#vectorize labels
vectorized_label_list = []
for sentence in new_label_list:
    sentence_vector = []
    for word in sentence:
        vector = [0]*len(label_words_to_index)
        vector[word] = 1
        sentence_vector.append(vector)
    vectorized_label_list.append(sentence_vector)
    
label_array = np.asarray(vectorized_label_list)

In [13]:
print(label_array.shape)
print(len(label_words_to_index))
print(len(label_index_to_words))

(79, 50, 12)
12
12


In [14]:
def logits_to_text(logits, dictionary):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """    
    return ' '.join([label_index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [15]:
def cascaded_reasoning(info_shape, questions_shape, facts_shape \
                       , relations_shape, labels_index_len):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Implement
    learning_rate = 0.01
    hidden_units = 128
    latent_dim = 128 
    # TODO: Build the layers
    
    LSTM_info_inputs = Input(shape=(None, info_shape[2]))
    LSTM_info = LSTM(hidden_units,return_state=True)
    LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
    info_states = [info_h,info_c]
    
    LSTM_questions_inputs = Input(shape=(None, questions_shape[2]))
    LSTM_questions = LSTM(hidden_units,return_state=True)
    LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
    questions_states= [questions_h,questions_c]
    
    LSTM_facts_inputs = Input(shape=(None, facts_shape[2]))
    LSTM_facts = LSTM(hidden_units,return_state=True)
    LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
    facts_states= [facts_h,facts_c]
    
    LSTM_rela_inputs = Input(shape=(None, relations_shape[2]))
    LSTM_rela = LSTM(hidden_units,return_state=True)
    LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
    rela_states= [rela_h,rela_c]
    
    added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
    added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
    states = [added_h,added_c]
    
    decoder_inputs = Input(shape=(1, labels_index_len))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = Dense(labels_index_len, activation='softmax')

    all_outputs = []
    inputs = decoder_inputs
    for _ in range(label_pad_up_to):
        # Run the decoder on one timestep
        outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
        outputs = decoder_dense(outputs)
        # Store the current prediction (we will concatenate all predictions later)
        all_outputs.append(outputs)
        # Reinject the outputs as inputs for the next loop iteration
        # as well as update the states
        inputs = outputs
        states = [state_h, state_c]

    # Concatenate all predictions
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

    # Define and compile model as previously
    model_T = Model([LSTM_info_inputs, LSTM_questions_inputs, LSTM_facts_inputs, \
                     LSTM_rela_inputs, decoder_inputs], decoder_outputs)

    model_T.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model_T

#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Trees are high."  Label result = "Trees -> High"
#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Pick apples from tree."  Label result = "Apples -> On tree"
#Example inputs: X1 = "Apples->On tree, Tree->High" X2 = "What do you need to get the apples?"
# X3 = "You can go on tree with a ladder."  Label result = "Ladder"

In [16]:
#tests.test_model_final(model_final)
batch_size = 79

print(len(label_index_to_words))

cascaded_reasoning= cascaded_reasoning(vectorized_info.shape, vectorized_questions.shape, \
                       vectorized_fact_list.shape, vectorized_relations.shape, len(label_index_to_words))

print('cascaded_reasoning loaded')

# Prepare decoder input data that just contains the start character
# Note that we could have made it a constant hard-coded in the model
decoder_input_data = np.zeros((len(new_facts_list), 1, len(label_index_to_words)))
decoder_input_data[:, 0, label_words_to_index['\t']] = 1.

cascaded_reasoning.fit([vectorized_info, vectorized_questions, \
                        vectorized_fact_list, vectorized_relations, decoder_input_data], \
                       label_array, batch_size=batch_size, epochs=50, validation_split=0.1)

print("model fit")



12
cascaded_reasoning loaded
Train on 71 samples, validate on 8 samples
Epoch 1/50
71/71 [==============================] - 71s 1s/step - loss: 2.5034 - acc: 0.0051 - val_loss: 2.3798 - val_acc: 0.9600
Epoch 2/50
71/71 [==============================] - 4s 59ms/step - loss: 2.3812 - acc: 0.9428 - val_loss: 2.2177 - val_acc: 0.9600
Epoch 3/50
71/71 [==============================] - 4s 54ms/step - loss: 2.2214 - acc: 0.9428 - val_loss: 1.5706 - val_acc: 0.9600
Epoch 4/50
71/71 [==============================] - 4s 54ms/step - loss: 1.5825 - acc: 0.9428 - val_loss: 0.2293 - val_acc: 0.9600
Epoch 5/50
71/71 [==============================] - 4s 55ms/step - loss: 0.3024 - acc: 0.9428 - val_loss: 0.1804 - val_acc: 0.9600
Epoch 6/50
71/71 [==============================] - 4s 54ms/step - loss: 0.2189 - acc: 0.9428 - val_loss: 0.1699 - val_acc: 0.9675
Epoch 7/50
71/71 [==============================] - 4s 55ms/step - loss: 0.1935 - acc: 0.9555 - val_loss: 0.1493 - val_acc: 0.9600
Epoch 8/50
7

In [17]:
hidden_units = 128
latent_dim = 128 
labels_index_len = len(label_index_to_words)
    
LSTM_info_inputs = Input(shape=(None, vectorized_info.shape[2]))
LSTM_info = LSTM(hidden_units,return_state=True)
LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
info_states = [info_h,info_c]
    
LSTM_questions_inputs = Input(shape=(None, vectorized_questions.shape[2]))
LSTM_questions = LSTM(hidden_units,return_state=True)
LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
questions_states= [questions_h,questions_c]
    
LSTM_facts_inputs = Input(shape=(None, vectorized_fact_list.shape[2]))
LSTM_facts = LSTM(hidden_units,return_state=True)
LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
facts_states= [facts_h,facts_c]
    
LSTM_rela_inputs = Input(shape=(None, vectorized_relations.shape[2]))
LSTM_rela = LSTM(hidden_units,return_state=True)
LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
rela_states= [rela_h,rela_c]
    
added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
states = [added_h,added_c]

encoder_model = Model([LSTM_info_inputs,LSTM_questions_inputs,LSTM_facts_inputs,LSTM_rela_inputs],states)

fact_model = Model(LSTM_facts_inputs, facts_states)
ques_model = Model(LSTM_questions_inputs, questions_states)
info_model = Model(LSTM_info_inputs, info_states)
rela_model = Model(LSTM_rela_inputs, rela_states)

decoder_inputs = Input(shape=(1, labels_index_len))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(labels_index_len, activation='softmax')

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [18]:
def decode_sequence(info_input, facts_input, ques_input, rela_input,label_words_to_index=label_words_to_index, \
                                        label_index_to_words=label_index_to_words, label_pad_up_to=label_pad_up_to):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict([info_input, facts_input, ques_input, rela_input])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(label_words_to_index)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, label_words_to_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = label_index_to_words[sampled_token_index]
        decoded_sentence += sampled_char + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > label_pad_up_to):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(label_words_to_index)))
        target_seq[0, 0, label_words_to_index['\t']] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
test_info =  vectorized_info[0,:,:]
test_info = test_info.reshape(1,*test_info.shape) 
test_fact = vectorized_fact_list[0,:,:]
test_fact = test_fact.reshape(1,*test_fact.shape)
test_ques = vectorized_questions[0,:,:]
test_ques = test_ques.reshape(1,*test_ques.shape)
test_rela = vectorized_relations[0,:,:]
test_rela = test_rela.reshape(1,*test_rela.shape)

decoded_sentence = decode_sequence(test_info, test_fact, test_ques, test_rela)

In [20]:
print(decoded_sentence)

IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 IN2 
